## pico_check.ipynb
This is a notebook to read and process data from our Picoscope. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pico_readout import PicoReadout

In [ ]:
np.set_printoptions(suppress=True) # to make tiny values appear as zero
np.set_printoptions(precision=4)   # number of digits in floats (or longfloats)
#np.set_printoptions(formatter={'float_kind':"{:.2f}"})
#np.set_printoptions()

%matplotlib inline

# Set formats
DARK=False
if(DARK==False):
    form='.k'
    plt.style.use(['default']) 
    fontcolor='k'
else:
    form='.w'
    plt.style.use(['dark_background'])
    fontcolor='w'


SMALL_SIZE = 18
MEDIUM_SIZE = 20
BIGGER_SIZE = 22

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

params = {
    'font.family': 'serif',
    'font.size' : 14, 'axes.titlesize' : 14, 'axes.labelsize' : 14, 'axes.linewidth' : 1,
    # ticks
    'xtick.labelsize' : 14, 'ytick.labelsize' : 14, 'xtick.major.size' : 8, 'xtick.minor.size' : 4,
    'ytick.major.size' : 8, 'ytick.minor.size' : 4, 'xtick.major.width' : 1, 'xtick.minor.width' : 1,
    'ytick.major.width' : 1, 'ytick.minor.width' : 1, 'xtick.direction' : 'in', 'ytick.direction' : 'in',
    # markers
    'lines.markersize' : 4, 'lines.markeredgewidth' : 1, 'errorbar.capsize' : 5, 'lines.linewidth' : 1.5,
    #'lines.linestyle' : None, 'lines.marker' : None,
    'savefig.bbox' : 'tight', 'legend.fontsize' : 14,
    'axes.labelsize': 14, 'axes.titlesize':16, 'xtick.labelsize':14, 'ytick.labelsize':14,
    'backend': 'Agg', 'mathtext.fontset': 'dejavuserif',
    'figure.facecolor':'w',
    #pad
    'axes.labelpad':8,
    # ticks
    'xtick.major.pad': 7,   'xtick.minor.pad': 7,   
    'ytick.major.pad': 4, 'ytick.minor.pad': 4,
}
plt.rcParams.update(params)

### Picoscope setup

In [ ]:
### available values are 'DC' and 'AC'
ADecouple = 'AC'
BDecouple = 'AC'

### available values are 20, 10, 5, 2, 1, 0.5, 0.2, 0.1, 0.05, and 0.02, in Volts
ARange = 1
BRange = .02

### name
name = 'CuNi'

### Temperature in Celsius
temperature = 25

### Scale or not
scale = True

### in seconds
min_collection_time = 20e-3

### Collect data from Picoscope

In [ ]:
data = PicoReadout(ADecouple, BDecouple, ARange, BRange, min_collection_time)
print(data)

In [ ]:
times, voltages, overflow_warnings = data.run(data.ADecouple, data.BDecouple, data.ARange, data.BRange, data.min_collection_time)

### Waveform

In [ ]:
t = times[::1000]
v_a = voltages["A"][::1000]
v_b = voltages["B"][::1000]

In [ ]:
if name == 'mild steel':
    area = (3.014e-3 / 2)**2 * np.pi
    volume = 101.22e-3 * (3.014e-3 / 2)**2 * np.pi
elif name == 'transformer iron':
    area = 0.6134e-3 * 4.30e-3
    volume = 0.6134e-3 * 85.74e-3 * 4.30e-3
elif name == 'CuNi':
    area = (5.000e-3 / 2)**2 * np.pi
    volume = 73.86e-3 * (5.000e-3 / 2)**2 * np.pi
else:
    print('unknown material')

if scale:

    ### in Tesla
    Bfield = 1e-5 * v_b * (20000 / np.max(3944.77 * v_a)) / area
    ### in A/m
    Hfield = 3944.77 * v_a * (20000 / np.max(3944.77 * v_a))
elif not scale:
    ### in Tesla
    Bfield = 0.25116 * v_b
    ### in A/m
    Hfield = 3944.77 * v_a

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('Hysteresis Loop of '+ name)
plt.plot(Hfield, Bfield, color='royalblue')
plt.ylabel(r'$B$ [T]')
plt.xlabel(r'$H$ [A/m]')
plt.grid(True)
plt.savefig(name+'.png')
plt.show()

### Hysteresis Area

In [ ]:
def hysteresis_area(x, y):
    """
    Calculate the area enclosed by the hysteresis loop using the trapezoidal rule.
    
    Parameters:
    - x: Array of x-values (e.g., independent variable)
    - y: Array of y-values (e.g., dependent variable)

    Returns:
    - area: The area of the loop (absolute value).
    """
    
    # Ensure the loop is closed by appending the first point at the end
    x_closed = np.append(x, x[0])
    y_closed = np.append(y, y[0])
    
    # Calculate the area using the trapezoidal rule
    area = np.trapz(y_closed, x_closed)
    
    return abs(area)

hisarea = hysteresis_area(Hfield, Bfield)
print(hisarea, 'J per unit volume per cycle round')
print(hisarea * 50, 'J per unit volume per second')
print(hisarea * volume, 'J per cycle round')
print(hisarea * 50 * volume, 'W')

### Generating dataframe

In [ ]:
### create dataframes for A and B channel data
### time in seconds, A and B channel data in volts
df = pd.DataFrame({"time": t, "A": v_a, "B": v_b})

### run if you want to save the data to a csv file
# df.to_csv('my_data.csv')

In [ ]:
### check the dataframe
df